## Imports

In [2]:
import pandas as pd

from oauthlib.oauth2 import WebApplicationClient
from defs import get_data_strava, request_token

## Code

### Authentication

In [3]:
access_token = request_token()

Requesting Token...



In [4]:
access_token

'595b13cadcdced379b4dabb36de42f5ecf5f8e3c'

### Get athlete data

In [16]:
# GET request to Strava API
url = 'https://www.strava.com/api/v3/athlete'
athlete = get_data_strava(url)

In [17]:
athlete

{'id': 30297538,
 'username': 'matheus_orsi',
 'resource_state': 2,
 'firstname': 'Matheus',
 'lastname': 'Orsi',
 'bio': '@matheus.orsi',
 'city': 'Campinas',
 'state': 'São Paulo',
 'country': 'Brazil',
 'sex': 'M',
 'premium': False,
 'summit': False,
 'created_at': '2018-04-29T21:37:16Z',
 'updated_at': '2023-10-07T02:59:11Z',
 'badge_type_id': 0,
 'weight': 76.0,
 'profile_medium': 'https://graph.facebook.com/1682174085195768/picture?height=256&width=256',
 'profile': 'https://graph.facebook.com/1682174085195768/picture?height=256&width=256',
 'friend': None,
 'follower': None}

In [12]:
athlete


{'id': 30297538,
 'username': 'matheus_orsi',
 'resource_state': 2,
 'firstname': 'Matheus',
 'lastname': 'Orsi',
 'bio': '@matheus.orsi',
 'city': 'Campinas',
 'state': 'São Paulo',
 'country': 'Brazil',
 'sex': 'M',
 'premium': False,
 'summit': False,
 'created_at': '2018-04-29T21:37:16Z',
 'updated_at': '2023-10-07T02:59:11Z',
 'badge_type_id': 0,
 'weight': 76.0,
 'profile_medium': 'https://graph.facebook.com/1682174085195768/picture?height=256&width=256',
 'profile': 'https://graph.facebook.com/1682174085195768/picture?height=256&width=256',
 'friend': None,
 'follower': None}

In [ ]:
df_ids = pd.DataFrame(columns=['id','username',])

In [8]:
# Get athlete id and pull stats
athlete_id = athlete['id']
url = f'https://www.strava.com/api/v3/athletes/{athlete_id}/stats'
stats = get_data_strava(url)

404 Client Error: Not Found for url: https://www.strava.com/api/v3/athletes/9978483361/stats


In [6]:





stats

{'biggest_ride_distance': 67314.6,
 'biggest_climb_elevation_gain': 34.80000000000007,
 'recent_ride_totals': {'count': 10,
  'distance': 187045.94140625,
  'moving_time': 39930,
  'elapsed_time': 41158,
  'elevation_gain': 2269.0770721435547,
  'achievement_count': 198},
 'all_ride_totals': {'count': 199,
  'distance': 3546913,
  'moving_time': 586827,
  'elapsed_time': 670227,
  'elevation_gain': 50747},
 'recent_run_totals': {'count': 0,
  'distance': 0.0,
  'moving_time': 0,
  'elapsed_time': 0,
  'elevation_gain': 0.0,
  'achievement_count': 0},
 'all_run_totals': {'count': 5,
  'distance': 38993,
  'moving_time': 16997,
  'elapsed_time': 28539,
  'elevation_gain': 324},
 'recent_swim_totals': {'count': 0,
  'distance': 0.0,
  'moving_time': 0,
  'elapsed_time': 0,
  'elevation_gain': 0.0,
  'achievement_count': 0},
 'all_swim_totals': {'count': 0,
  'distance': 0,
  'moving_time': 0,
  'elapsed_time': 0,
  'elevation_gain': 0},
 'ytd_ride_totals': {'count': 25,
  'distance': 7221